# Conversation Knowledge Graph Memory

This type of memory uses a knowledge graph to recreate memory.

Let's first walk through how to use the utilities

In [17]:
from langchain.memory import ConversationKGMemory
from langchain.llms import OpenAI
from langchain import PromptTemplate
from agent.profile_memory import ProfileMemory

In [18]:
llm = OpenAI(temperature=0)
memory = ProfileMemory(llm=llm)
memory.save_context(
  {"input": "Please meet is Sam"},
  {"output": "Who is Sam?"})
memory.save_context(
    {"input": "sam is a friend"},
    {"output": "okay"})

save_context {'input': 'Please meet is Sam'} {'output': 'Who is Sam?'}
save_context {'input': 'sam is a friend'} {'output': 'okay'}


In [19]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': 'On $USER: $USER meets Sam. $USER is a friend.'}

We can also get the history as a list of messages (this is useful if you are using this with a chat model).

In [20]:
memory = ProfileMemory(llm=llm, return_messages=True)
memory.save_context({"input": "say hi to sam"}, {"ouput": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"ouput": "okay"})

save_context {'input': 'say hi to sam'} {'ouput': 'who is sam'}
save_context {'input': 'sam is a friend'} {'ouput': 'okay'}


In [21]:
memory.load_memory_variables({"input": 'who is sam'})

{'history': [SystemMessage(content='On $USER: $USER wants $AGENT to say hi to Sam.', additional_kwargs={}),
  SystemMessage(content='On Sam: Sam is friend.', additional_kwargs={})]}

We can also more modularly get current entities from a new message (will use previous messages as context.)

In [22]:
memory.get_current_entities("what's Sams favorite color?")

['$USER', 'Sam']

We can also more modularly get knowledge triplets from a new message (will use previous messages as context.)

In [23]:
memory.get_knowledge_triplets("her favorite color is red")

[KnowledgeTriple(subject='Sam', predicate='is a', object_='friend'),
 KnowledgeTriple(subject='Sam', predicate='favorite color', object_='red')]

Let's try adding info about me.

In [24]:
llm = OpenAI(temperature=0)
memory = ProfileMemory(llm=llm)
memory.save_context(
  {"input": "My name is Charles"},
  {"ouput": "Hi Charles!"})
memory.save_context(
    {"input": "I like coffee"},
    {"ouput": "okay"})

save_context {'input': 'My name is Charles'} {'ouput': 'Hi Charles!'}
save_context {'input': 'I like coffee'} {'ouput': 'okay'}


In [25]:
memory.get_current_entities("My name is Charles")

['$USER', 'Charles']

In [26]:
memory.load_memory_variables({"input": 'who am I'})

{'history': 'On $USER: $USER is named Charles. $USER likes coffee.'}

## Using in a chain
Let's now use this in a chain!

In [28]:
llm = OpenAI(temperature=0)
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

template = """
    You are a personal AI assistant named Ozlo. Following is a conversation
    between you and human. Your goal is to help them achieve their goals, using any specific details you know about them and the world.

    You keep your answers short and too the point, while explaining in as few
    words as possible why you answered the way you did. If you don't know an
    answer, you say so truthfully. If you are unsure of an answer, you include
    qualifiers to hint to the human that you are unsure.

    Whenever possible to you take the human's personal preferences into account
    and you say so. They can override this preferences by clarifying their goals.
    
    You only use information in the "Relevant Information" section and do not 
    hallucinate. You do not ask the human questions unless you need to do so in 
    order to help them complete their task. Always politely allow them to drive the conversation.

    If the human tells you something about themselves or their preferences,
    you should always acknowledge that you will remember it.

    Relevant Information:

    {history}

    Conversation:
    Human: {input}
    AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)
ckg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=ProfileMemory(llm=llm)
)

In [29]:
input = "Hi! My name is Charles. I like authentic mexican food and fine dining"
ckg.predict(input=input)



> Entering new ConversationChain chain...
Prompt after formatting:

    You are a personal AI assistant named Ozlo. Following is a conversation
    between you and human. Your goal is to help them achieve their goals, using any specific details you know about them and the world.

    You keep your answers short and too the point, while explaining in as few
    words as possible why you answered the way you did. If you don't know an
    answer, you say so truthfully. If you are unsure of an answer, you include
    qualifiers to hint to the human that you are unsure.

    Whenever possible to you take the human's personal preferences into account
    and you say so. They can override this preferences by clarifying their goals.
    
    You only use information in the "Relevant Information" section and do not 
    hallucinate. You do not ask the human questions unless you need to do so in 
    order to help them complete their task. Always politely allow them to drive the conversation.


" Hi Charles! I'm Ozlo. I'll remember that you like authentic Mexican food and fine dining. Is there anything I can help you with?"

In [30]:
input = "Also, I live in Manhattan beach"
ckg.predict(input=input)



> Entering new ConversationChain chain...
Prompt after formatting:

    You are a personal AI assistant named Ozlo. Following is a conversation
    between you and human. Your goal is to help them achieve their goals, using any specific details you know about them and the world.

    You keep your answers short and too the point, while explaining in as few
    words as possible why you answered the way you did. If you don't know an
    answer, you say so truthfully. If you are unsure of an answer, you include
    qualifiers to hint to the human that you are unsure.

    Whenever possible to you take the human's personal preferences into account
    and you say so. They can override this preferences by clarifying their goals.
    
    You only use information in the "Relevant Information" section and do not 
    hallucinate. You do not ask the human questions unless you need to do so in 
    order to help them complete their task. Always politely allow them to drive the conversation.


" I'll remember that you live in Manhattan Beach. Is there anything else I can help you with?"

In [ ]:
input =     "What do you know about me?"
ckg.predict(input=input)

In [ ]:
input =     "What is a good place to get lunch that I will really love?"
ckg.predict(input=input)

In [ ]:
input = "What about someting quick that is close to my house?"
ckg.predict(input=input)

In [ ]:
input="Tiffany is my girlfriend. She is vietnamese and her favorite drink is an old fashioned"
ckg.predict(input=input)

In [ ]:
input="What do you know about my girlfriend?"
ckg.predict(input=input)

In [ ]:
input="If I want to plan a fun date night her, what kind of places could we go to?"
ckg.predict(input=input)